In [112]:
from datetime import datetime
import pandas as pd
import pickle

In [113]:
class Trade:
    def __init__(self, stock_symbol, trade_type, quantity, price, trade_date):
        self.stock_symbol = stock_symbol
        self.trade_type = trade_type  # "buy" or "sell"
        self.quantity = quantity
        self.price = price
        self.trade_date = trade_date

    def __str__(self):
        return f"{self.trade_type.title()} {self.quantity} of {self.stock_symbol} at ${self.price} each on {self.trade_date}"

In [116]:
class TradeManager:
    def __init__(self, filename='trades.xlsx'):
        self.filename = filename
        self.trades = self.load_trades()

    def add_trade(self, trade):
        self.trades.append(trade)
        self.save_trades()

    def trade_dict(self):
        return {
            "Stock Symbol": [trade.stock_symbol for trade in self.trades],
            "Trade Type": [trade.trade_type for trade in self.trades],
            "Quantity": [trade.quantity for trade in self.trades],
            "Price": [trade.price for trade in self.trades],
            "Trade Date": [trade.trade_date for trade in self.trades]
        }

    def save_trades(self):
        df = pd.DataFrame(self.trade_dict())
        df.to_excel(self.filename, index=False)
    
    def sum_trades(self):
        temp = pd.read_excel('trades.xlsx')
        temp['Symbol2'] = temp['Stock Symbol'].str.upper()
        temp.loc[temp['Trade Type'].str.upper() == "BUY", 'QT2'] = temp['Quantity']
        temp.loc[temp['Trade Type'].str.upper() == "SELL", 'QT2'] = -temp['Quantity']
        temp['cost'] = temp['Price'] * temp['QT2']
        result = temp.groupby('Symbol2').agg({
         'QT2': ['sum'],  # Sum and mean of quantities sold
         'cost': ['sum']  # Sum, mean, and maximum of revenue
         })
        for k in result[('QT2', 'sum')]:
            if k > 0:
                result['AVG']=round(result[('cost', 'sum')]/result[('QT2', 'sum')],2)
            else:
                result['AVG']=999
                
        result.to_excel('Trades_AVG.xlsx', index=True ,engine='openpyxl')
        
        
    def load_trades(self):
        try:
            df = pd.read_excel(self.filename)
            trades = [Trade(row['Stock Symbol'], row['Trade Type'], row['Quantity'], row['Price'], row['Trade Date']) for index, row in df.iterrows()]
            return trades
        except FileNotFoundError:
            return []

    def display_trades(self):
        if not self.trades:
            print("No trades recorded.")
            return
        for trade in self.trades:
            print(trade)

In [117]:
def main():
    trade_manager = TradeManager()
    while True:
        print("\nStock Trade Recorder")
        print("1. Record a new trade")
        print("2. Display all trades")
        print("3. Display trade summary")
        print("4. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            stock_symbol = input("Enter stock symbol: ")
            trade_type = input("Enter trade type (buy/sell): ")
            quantity = float(input("Enter quantity: "))
            price = float(input("Enter price per unit: "))
            trade_date = input("Enter trade date (YYYY-MM-DD): ") or datetime.now().strftime("%Y-%m-%d")

            trade = Trade(stock_symbol, trade_type, quantity, price, trade_date)
            trade_manager.add_trade(trade)
            print("Trade recorded successfully.")

        elif choice == '2':
            print("\nRecorded Trades:")
            trade_manager.display_trades()
            
        elif choice == '3':
            print("\nSummary Output:")
            trade_manager.sum_trades()

        elif choice == '4':
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Stock Trade Recorder
1. Record a new trade
2. Display all trades
3. Display trade summary
4. Exit
Enter choice: 1
Enter stock symbol: TMF
Enter trade type (buy/sell): buy
Enter quantity: 90.54
Enter price per unit: 53.66
Enter trade date (YYYY-MM-DD): 2024-03-28
Trade recorded successfully.

Stock Trade Recorder
1. Record a new trade
2. Display all trades
3. Display trade summary
4. Exit
Enter choice: 1
Enter stock symbol: TMF
Enter trade type (buy/sell): sell
Enter quantity: 55
Enter price per unit: 47.49
Enter trade date (YYYY-MM-DD): 2024-03-28
Trade recorded successfully.

Stock Trade Recorder
1. Record a new trade
2. Display all trades
3. Display trade summary
4. Exit
Enter choice: 3

Summary Output:

Stock Trade Recorder
1. Record a new trade
2. Display all trades
3. Display trade summary
4. Exit
Enter choice: 1
Enter stock symbol: PDD
Enter trade type (buy/sell): buy
Enter quantity: 10
Enter price per unit: 113.6
Enter trade date (YYYY-MM-DD): 2024-04-15
Trade recorded successf